In [21]:
# Import necessary libraries

import requests
import lxml.html as lh
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#Create a handle, page, to handle the contents of the website
page = requests.get(url)

#Store the contents of the website under doc
doc = lh.fromstring(page.content)

#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [3]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

There are three columns per row

In [4]:

# Parse the first row as our header
tr_elements = doc.xpath('//tr')

#Create empty list
col=[]
i=0

#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))



1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


Creating a Pandas dataframe

Making a header appended to a tuple along with an empty list.

In [5]:


#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1



In [6]:
# Check the length of each column. Ideally, they should all be the same
[len(C) for (title,C) in col]

[181, 181, 181]

Each column has 181 rows

Creating a pandas dataframe

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
# Access the top 5 rows of the data frame 
df.head()



,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Re arranging and naming columns

In [11]:
df.columns = ['Borough', 'Neighbourhood','Postcode']

cols = df.columns.tolist()
cols

cols = cols[-1:] + cols[:-1]

df = df[cols]

df.head()



,Postcode,Borough,Neighbourhood
0,Not assigned\n,Not assigned\n,M1A\n
1,Not assigned\n,Not assigned\n,M2A\n
2,North York\n,Parkwoods\n,M3A\n
3,North York\n,Victoria Village\n,M4A\n
4,Downtown Toronto\n,"Regent Park, Harbourfront\n",M5A\n


Cleaning the messy string in the column of Borough

In [12]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A


Dropping all cells with a borough that is not assigned

In [13]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)

# Reset the index and dropping the previous index
df = df.reset_index(drop=True)

df.head(10)

,Postcode,Borough,Neighbourhood
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A
5,North York,"Lawrence Manor, Lawrence Heights",M6A
6,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A
7,Not assigned,Not assigned,M8A
8,Etobicoke,"Islington Avenue, Humber Valley Village",M9A
9,Scarborough,"Malvern, Rouge",M1B


Combining neighbourhoods with a similar postcode and borough

In [14]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Canadian postal codes,,
1,Central Toronto,Davisville,M4S
2,Central Toronto,Davisville North,M4P
3,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",M5P
4,Central Toronto,Lawrence Park,M4N
5,Central Toronto,"Moore Park, Summerhill East",M4T
6,Central Toronto,"North Toronto West, Lawrence Park",M4R
7,Central Toronto,Roselawn,M5N
8,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",M4V
9,Central Toronto,"The Annex, North Midtown, Yorkville",M5R


Removing any space at the beginning of a string

In [15]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

Assigning Borough values to the Neignbourhood where vlaue is "Not assigned"

In [16]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [18]:
# Check if the Neighbourhood for Queen's Park changed 
df[df['Borough'] == 'Queen\'s Park']



,Postcode,Borough,Neighbourhood


In [19]:
# Check the shape of the data frame
df.shape

(101, 3)

Saving this to a CSV file

In [20]:
df.to_csv(r'df_can.csv')

